In [ ]:
#In windows, run anaconda prompt, and go to BIDS folder. Run jupyter notebook
#MAIN CODE TO CALCULATE SEED BASED VOXEL IN BATCH MODE FOLLOWING BIDS AND fMRIPREP

import numpy as np
import os, sys
import bids
import nibabel as nib
from scipy.stats import pearsonr
import statsmodels.api as sm
from nilearn import plotting as nplot
from nilearn import image as nimg
from nilearn.image import resample_to_img
import matplotlib.pyplot as plt
import argparse
import pandas as pd
#%%
##
def parse():

        options = argparse.ArgumentParser(description="Run 1st level analysis. Created by ...")
        options.add_argument('-p', '--participants', nargs='+',dest="participants", action='store', type=str, required=False,
                            help='id of subject or list of subjects')
        options.set_defaults(participants=None)
        options.add_argument('-w', '--workdir',dest="workdir", action='store', type=str, required=False,
                            help='the work directory for the project')
        options.set_defaults(workdir=os.environ["ROOTDIR"])
        options.add_argument('-b', '--bidsdir',dest="rawdata", action='store', type=str, required=False,
                            help='the work directory for the project')
        options.set_defaults(rawdata=os.path.join(os.environ["ROOTDIR"],"BIDS"))
        options.add_argument('-d', '--derivatives',dest="derivatives", action='store', type=str, required=True,
                            help='path to fMRIprep directory')
        options.add_argument('-o', '--outputs',dest="output", action='store', type=str, required=True,
                            help='path to fMRIprep directory')
        #print(options.parse_args())

        return options.parse_args()

#%%
def main ():
    os.environ["ROOTDIR"] = r'C:\Users\angel\Documents\INPD_neuroimages_CAPE'  # seth path
    rootdir = os.environ["ROOTDIR"]
    if hasattr(sys, "ps1"):
        options = {}
        workdir = os.environ["ROOTDIR"]
        rawdata = os.path.join(workdir,"BIDS")
        derivat = os.path.join(workdir,"BIDS","derivatives","fmriprep")
        output  = os.path.join(workdir,"BIDS","derivatives","BOLD_ROI")
        masks = os.path.join(workdir,"analisys_code","atlas")
        participants = []

    else :
        options = parse()
        participants = options.participants
        workdir = options.workdir
        rawdata = options.rawdata
        derivat = options.derivatives
        output  = options.output
        script_path = os.path.dirname(__file__)
        masks = os.path.join(script_path,"masks")

    with open("ROI_names.txt", "r", encoding="utf-16") as file:
        seednames = [line.strip() for line in file if line.strip()]

    print('Rawdata: ', rawdata)

    bidslayout = bids.BIDSLayout(derivat,derivatives=True, validate = False) #With validate = True it doesn't find any subjects

    if not participants:
        participants = bidslayout.get_subjects()
    for i in  participants:
        print("Subject: ", i)
        i = i.replace("sub-", "")
        for ses in bidslayout.get_sessions(subject=i):
            print("Session: ", ses)
            #for multi-echo files, apply transform to standard space
            for r in bidslayout.get_runs(subject=i, session=ses):
                print("Run:", r)
                filesrest = bidslayout.get(subject=i,
                                 session=ses,
                                 run=r,
                                 extension=".nii.gz",
                                 suffix="bold",
                                 space="MNI152NLin2009cAsym",
                                 #regex_search=True,
                                 #invalid_filters="allow"
                                 )

                print("Filerest:", filesrest)
                if len(filesrest)>1:
                    print(f"More than one file match bold filters. Subject {i}, ses {ses}, run {r}")
                    continue
                elif len(filesrest)<1:
                    print(f"No file match bold filters. Subject {i}, ses {ses}, run {r}")
                    continue
                bold = filesrest[0]

                filesbrainmask = bidslayout.get(subject=i,
                                  session=ses,
                                  run=r,
                                  task='rest',
                                  extension=".nii.gz",
                                  suffix="mask",
                                  space="MNI152NLin2009cAsym",
                                  #regex_search=True,
                                  invalid_filters="allow"
                                  )
                print("Brain_Mask:", filesbrainmask)
                if len(filesbrainmask)>1:
                    print(f"More than one file match brainmask filters. Subject {i}, ses {ses}, run {r}")
                    continue
                brainmask = filesbrainmask[0]


                filesanat = bidslayout.get(subject=i,
                                 session=ses,
                                 run=r,
                                 extension=".nii.gz",
                                 suffix="T1w",
                                 #desc="preproc",
                                 space="MNI152NLin2009cAsym",
                                 #regex_search=True,
                                 invalid_filters="allow"
                                 )

                print(f"Anat: {filesanat} ")
                if len(filesrest)>1:
                    print(f"More than one file match anat filters. Subject {i}, ses {ses}, run {r}")
                    continue
                
                #Load Confounds
                confounds_ents = {}
                confounds_ents["subject"] = bold.entities['subject']
                confounds_ents["session"] = bold.entities['session']
                #confounds_ents["acquisition"] = bold.entities['acquisition']
                confounds_ents["run"] = bold.entities['run']
                confounds_ents["task"] = bold.entities['TaskName']
                #confounds_ents['desc'] = "confounds"
                confounds_ents['suffix'] = "timeseries"
                confounds_ents['extension'] = ".tsv"
                confounds = bidslayout.get(return_type='file', **confounds_ents, invalid_filters="allow")[0]
                allconfounds_df = pd.read_csv(confounds, sep='\t')

                melodic_ents = confounds_ents
                melodic_ents['suffix'] = "mixing"
                melodic = bidslayout.get(return_type='file', **melodic_ents, invalid_filters="allow")[0]
                melodic_df = pd.read_csv(melodic, sep='\t')

                aroma_noise = confounds_ents
                aroma_noise['suffix'] = "AROMAnoiseICs"
                aroma_noise['extension'] = ".csv"
                aroma_noise = bidslayout.get(return_type='file', **aroma_noise, invalid_filters="allow")[0]
                aroma_noise_idx = np.genfromtxt(aroma_noise, delimiter=',').astype(int)

                aroma_conf = melodic_df.iloc[:,aroma_noise_idx-1]

                conf_index= ['csf', 'white_matter','cosine00','cosine01','cosine02','cosine03']
                confounds_df = allconfounds_df[conf_index]

                #DROP 4 initial volumes and generate confounds matrix
                drop_confound_df = confounds_df.loc[4:].values #from confounds
                drop_aroma_conf = aroma_conf.loc[3:].values #it has one less time point

                # try:
                my_confounds = np.concatenate((drop_confound_df, drop_aroma_conf), axis=1)

                #DROP 4 volumes from bold data
                raw_func_img = nimg.load_img(bold)
                func_img = raw_func_img.slicer[:,:,:,4:]
                dim4d = func_img.shape
                N = dim4d[-1]  # number of time points

                # #reshape into 2d
                func_2d = func_img.get_fdata().reshape(-1, func_img.shape[-1]).T

                # pre-allocate space for seeds' time series
                TS_seeds = np.empty((N, len(seednames)))
                # Input new nifti for seeds
                # Compute SEEDs TS
                for k in range(len(seednames)):
                    print('----- TS Seed:', seednames[k], '-----')

                    seed = os.path.join(masks, seednames[k])
                    #load seed image
                    Vseed_img = nib.load(seed)
                    Vseed = Vseed_img.get_fdata().reshape(func_2d.shape[-1])

                    #for each time point, compute mean of voxels in which seed==1
                    for vol in range(N):
                        Vaux = func_2d[vol, :]
                        TS_seeds[vol, k] = np.mean(Vaux[Vseed == 1])
                        del Vaux

                    del Vseed

                '''Just to have it clear, the TS_seeds have the following structure:
                TS_seeds = [PARCELATIONS_ROIS, TSseed1, TSseed2, TSseed3, TSseed4, TSseed5, TSseed6]
                '''

                #--------- 1st Level Analysis: -------------------------------------------
                print('First level analysis -----')
                print('TS_ROI = b0 + b_j*TSseed_j + CSF + WM + cosines + AROMA_noiseICs')
                t_map = np.zeros((len(seednames[:-6]), len(seednames[-6:])))  # t[ROIs,seeds]

                # sinal = np.reshape(TS_seeds[:,-6:], (len(TS_seeds[:,-6:]),1))
                # design_matrix = np.concatenate((sinal,my_confounds), axis=1)
                # design_matrix = sm.add_constant(design_matrix)
                # print('Design matrix shape:', design_matrix.shape)

                for roi in range(len(seednames[:-6])):
                    for seed in range(len(seednames[-6:])):
                        sinal = np.reshape(TS_seeds[:,seed], (len(TS_seeds[:,seed]),1))
                        design_matrix = np.concatenate((sinal,my_confounds), axis=1)
                        design_matrix = sm.add_constant(design_matrix)
                        print('Design matrix shape:', design_matrix.shape)

                        # Fit the multiple linear regression model
                        model = sm.OLS(TS_seeds[:, roi], design_matrix)
                        results = model.fit()

                        print('getting t-values for ROI:', seednames[roi], 'and seed:', seednames[(-1-seed)])
                        # Get the t-values
                        t_values = results.tvalues
                        t_map[roi, :] = t_values[1:7] #t_values[0] is the constant and the others is confunds
                # ------------------------------------------------------------------------
                # addinig in the first line of t_map matrix the seed names
                # t_map = np.vstack((seednames[-6:], t_map))

                tmap_metadata = {
                    'task': 'rest',
                    'suffix': 'tstat',
                    'extension': '.txt',
                    'run': str(r),
                    'session': str(ses),
                    'subject': 'sub-'+i,
                    'space': 'MNI152NLin2009cAsym',
                    'seed': 'SeedtoROI'
                }

                # Save each tmap as a nifti file
                filepath = os.path.join(output,tmap_metadata["subject"],"ses-"+tmap_metadata['session'],"func")
                filename = tmap_metadata["subject"] + "_" + \
                            "ses-"+tmap_metadata['session'] + "_" + \
                            "task-"+tmap_metadata['task'] + "_" + \
                            "run-"+tmap_metadata['run'] + "_" + \
                            "space-"+tmap_metadata['space'] + "_" + \
                            "seed-"+tmap_metadata['seed'] + "_" + \
                            tmap_metadata["suffix"] + \
                            tmap_metadata["extension"]

                folder_subj=os.path.join(output,tmap_metadata["subject"])
                if not os.path.exists(folder_subj):
                    os.makedirs(folder_subj)

                if not os.path.exists(filepath):
                        os.makedirs(filepath)
                
                header = ' '.join(seednames[-6:])
                np.savetxt(os.path.join(filepath, filename), t_map, delimiter=' ', header=header, comments='')

                print('\t\t ----- Finished subject', i, ' Sess: ' ,ses,' Run: ',r,' ----')
                    
                # except:
                #     print('\t\t ----- ERRO NO SUJEITO', i, ' Sess: ' ,ses,' Run: ',r,' ----')
        
if __name__ == '__main__':
    main()

Rawdata:  C:\Users\angel\Documents\INPD_neuroimages_CAPE\BIDS


c:\Users\angel\Documents\INPD_neuroimages_CAPE\.venv_fmriINPD\Lib\site-packages\bids\layout\layout.py:516: UserWarning: Derivative indexing was requested, but no valid datasets were found in the specified locations ([WindowsPath('C:/Users/angel/Documents/INPD_neuroimages_CAPE/BIDS/derivatives/fmriprep/derivatives')]). Note that all BIDS-Derivatives datasets must meet all the requirements for BIDS-Raw datasets (a common problem is to fail to include a 'dataset_description.json' file in derivatives datasets).
Example contents of 'dataset_description.json':
{"Name": "Example dataset", "BIDSVersion": "1.0.2", "GeneratedBy": [{"Name": "Example pipeline"}]}
  warnings.warn("Derivative indexing was requested, but no valid "


Subject:  00003
Session:  1
Run: 1
Filerest: [<BIDSImageFile filename='C:\Users\angel\Documents\INPD_neuroimages_CAPE\BIDS\derivatives\fmriprep\sub-00003\ses-1\func\sub-00003_ses-1_task-rest_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz'>]
Brain_Mask: [<BIDSImageFile filename='C:\Users\angel\Documents\INPD_neuroimages_CAPE\BIDS\derivatives\fmriprep\sub-00003\ses-1\func\sub-00003_ses-1_task-rest_run-1_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz'>]
Anat: [<BIDSImageFile filename='C:\Users\angel\Documents\INPD_neuroimages_CAPE\BIDS\derivatives\fmriprep\sub-00003\ses-1\anat\sub-00003_ses-1_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_T1w.nii.gz'>] 
----- TS Seed: roi_0_mask-Schaefer2018-100Parcels7Networks.nii -----
----- TS Seed: roi_100_mask-Schaefer2018-100Parcels7Networks.nii -----
----- TS Seed: roi_10_mask-Schaefer2018-100Parcels7Networks.nii -----
----- TS Seed: roi_11_mask-Schaefer2018-100Parcels7Networks.nii -----
----- TS Seed: roi_12_mask-Schaef

In [ ]:
pd.read_csv(r'C:\Users\angel\Documents\INPD_neuroimages_CAPE\BIDS\derivatives\BOLD_ROI\sub-00003\ses-1\func\sub-00003_ses-1_task-rest_run-1_space-MNI152NLin2009cAsym_seed-SeedtoROI_tstat.txt', sep=' ')

,DCPutamen_space-MNI152NLin2009cAsym.nii,DRPutamen_space-MNI152NLin2009cAsym.nii,VRPutamen_space-MNI152NLin2009cAsym.nii,InfVentralCaudate_space-MNI152NLin2009cAsym.nii,SupVentralCaudate_space-MNI152NLin2009cAsym.nii,DorsalCaudate_space-MNI152NLin2009cAsym.nii
0,1.387781,5.166577,5.887719,5.840481,-0.686586,3.418319
1,1.611396,6.130344,1.326967,-2.952307,-1.518854,-3.135082
2,1.207885,-1.510647,1.113874,5.220452,4.153809,5.784169
3,1.022016,0.268063,0.079695,-4.039558,-0.174929,0.850445
4,4.597286,-2.826788,0.757187,3.632276,7.945582,6.143322
...,...,...,...,...,...,...
96,1.009992,5.487061,0.222060,-1.309722,1.932174,-2.104067
97,-0.998823,2.432729,-0.309793,2.400158,1.570800,-0.399964
98,0.325527,3.586249,3.255205,6.725259,0.307095,-0.008721
99,0.453840,3.442039,0.043436,-2.977743,-0.752570,-2.560551
